Урок 4. Тематическое моделирование. EM-алгоритм
Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

рабочие ссылки с твитами
https://disk.yandex.ru/i/v5HM-ENiGXZVpQ
https://disk.yandex.ru/i/koR5eMCToCZS2Q

как альтернатива можно скачать данные из Роспотребнадзора
https://zpp.rospotrebnadzor.ru/Forum/Appeals
для этого берём ноутбук parse_rospotrebnadzor.ipynb
устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать
1. объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
2. провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
3. сделать визуализацию кластеров тематик
4. проинтерпритировать получившиеся тематики

In [2]:
!pip install pymorphy2

In [3]:
!pip install stop_words

In [4]:
import pandas as pd
import numpy as np

import re

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words

from gensim import corpora
from gensim.models import *

In [5]:
morpher = MorphAnalyzer()

In [6]:
stopwords_list = set(get_stop_words("ru"))

In [7]:
positive = pd.read_csv('positive.csv', sep = ';', usecols = [3], names = ['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep = ';', usecols = [3], names = ['text'])
negative['label'] = ['negative'] * len(negative)

In [8]:
df = pd.concat([positive, negative], ignore_index = True)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226834 entries, 0 to 226833
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    226834 non-null  object
 1   label   226834 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [10]:
words_regex = re.compile(r'\w+')

# Формирование токенов длиной более 3 букв
def find_words(text, regex = words_regex):
    tokens = regex.findall(text) 
    return [w for w in tokens if w.isalpha() and len(w)>= 4] 


In [11]:
# Удаляем стоп слова и другие символы, не являющимися буквами
def lemmatize(words, lemmer = morpher, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words] 
    return [w for w in lemmas if not w in stopwords and w.isalpha()] 

#Чистим от нецензурных слов и слов, не несущих особой смысловой нагрузки
def clear_tokens(tokens):
    string_tokens = ' '.join(tokens)
    string_tokens = re.sub(r'[A-za-z]', ' ',  string_tokens) 
    string_tokens = re.sub(r'\w+ще\b', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bбл\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'значит', ' ',  string_tokens) 
    string_tokens = re.sub(r'почему', ' ',  string_tokens) 
    string_tokens = re.sub(r'свой', ' ',  string_tokens) 
    string_tokens = re.sub(r'капец', ' ',  string_tokens) 
    string_tokens = re.sub(r'который', ' ',  string_tokens)
    string_tokens = re.sub(r'это', ' ',  string_tokens)
    string_tokens = re.sub(r'весь', ' ',  string_tokens)
    string_tokens = re.sub(r'\bидио\w+', ' ',  string_tokens) 
    string_tokens = re.sub(r'\bпид\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхуй\w+', ' ', string_tokens)
    string_tokens = re.sub(r'\w+xу.\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+xу\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+но\b', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпоп\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпиз\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bгов\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bфиг\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'сука', ' ',  string_tokens)
    string_tokens = re.sub(r'очень', ' ',  string_tokens)
    string_tokens = re.sub(r'лох', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпи.де\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'нету', ' ',  string_tokens)
    string_tokens = re.sub(r'ой', ' ',  string_tokens)
    string_tokens = re.sub(r'\bхот\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bпрост\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\bника\w+', ' ',  string_tokens)
    string_tokens = re.sub(r'\w+ть\b', ' ',  string_tokens)
    string_tokens = re.sub(r'мочь', ' ',  string_tokens)
    clear_tokens = string_tokens.split()
    clear_tokens = [w for w in clear_tokens if len(w)>2]
    return clear_tokens

def preprocess(text):
    return (clear_tokens(lemmatize(find_words(text))))

In [12]:
df['text'] = df['text'].apply(preprocess)
df.head()

,text,label
0,"[школотый, поверь, самый, общество, предмет, тип]",positive
1,"[таки, похожий, мальчик, хороший]",positive
2,[испугаться],positive
3,"[угол, голод, порция]",positive
4,"[страшилка, создаться, ощущение, автор]",positive


In [13]:
# Словарь уникальных токенов
dictionary = corpora.Dictionary(df['text'])

In [14]:
# Выкидывание редких слов
dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None)
dictionary.save('tweet.dict')
len(dictionary.keys())

8100

In [15]:
# Векторизация твитов
corpus = [dictionary.doc2bow(text) for text in df['text']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [16]:
# Обработка словаря
mm = corpora.MmCorpus('tweet.model')
print(df['text'].values[0])
print(mm[0])

['школотый', 'поверь', 'самый', 'общество', 'предмет', 'тип']
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0)]


In [17]:
# Обучение модели
lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3)

In [18]:
# Определяем тематики
lda.show_topics(num_topics=3, num_words=20, formatted=True)

[(0,
  '0.019*"новый" + 0.016*"идти" + 0.016*"утро" + 0.012*"любимый" + 0.012*"голова" + 0.009*"телефон" + 0.009*"улица" + 0.009*"жалко" + 0.009*"прийтись" + 0.009*"нравиться" + 0.008*"снег" + 0.008*"бояться" + 0.008*"правда" + 0.007*"мама" + 0.006*"скоро" + 0.006*"фильм" + 0.006*"лента" + 0.005*"фотка" + 0.005*"прям" + 0.005*"добрый"'),
 (1,
  '0.022*"хороший" + 0.022*"больш" + 0.015*"никто" + 0.015*"жаль" + 0.013*"настроение" + 0.011*"самый" + 0.008*"казаться" + 0.007*"боль" + 0.007*"вчера" + 0.007*"ужасный" + 0.006*"город" + 0.006*"маленький" + 0.006*"бог" + 0.006*"выйти" + 0.006*"друг" + 0.006*"глаз" + 0.005*"найти" + 0.005*"получиться" + 0.005*"погода" + 0.005*"таки"'),
 (2,
  '0.031*"завтра" + 0.022*"дом" + 0.021*"школа" + 0.012*"час" + 0.011*"последний" + 0.010*"урок" + 0.010*"ночь" + 0.009*"неделя" + 0.009*"прийти" + 0.008*"остаться" + 0.008*"экзамен" + 0.008*"каникулы" + 0.007*"дело" + 0.007*"друг" + 0.007*"закончиться" + 0.006*"вроде" + 0.006*"твиттер" + 0.006*"работа" + 0.00

In [19]:
# Оценка качества модели
print(lda.log_perplexity(corpus))

-8.131030642196118


In [20]:
print('Персплексия: ', np.exp(lda.log_perplexity(corpus)))

Персплексия:  0.0002942647699411939


# Визуализация

In [21]:
!pip install pyLDAvis

In [22]:
import pyLDAvis.gensim_models as gensimvis

In [23]:
v_data = gensimvis.prepare(lda, corpus, dictionary)

/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/evgeniilukyanov/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/U

In [24]:
import pyLDAvis

In [25]:
pyLDAvis.display(v_data)

# Результаты:

У нас получились три частые темы:

1) Часть жизни, проходящая в работе или учебе. Т.е. трудовые будни.

2) Тема взаимоотношений между людьми.

3) Часть жизни не связанная с работой. Отдых и развлечения.